In [8]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import torch
from datasets import Dataset
from itertools import chain
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    LEDForConditionalGeneration,
    LEDTokenizer,
    Seq2SeqTrainingArguments,
    Trainer
)



In [9]:
!nvidia-smi


Sun Oct  6 17:55:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     Off |   00000000:01:00.0 Off |                    0 |
|  0%   61C    P0            211W /  300W |   17464MiB /  46068MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:


device = torch.device("cpu")
print(f"Using device: {device}")


Using device: cpu


In [11]:


# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    
    if num_gpus >= 3:  # Check if there are at least 3 GPUs
        # Set the device to GPU 2
        device = torch.device("cuda:2")
        print(f"Set device to: {device}")
        print(f"GPU Name: {torch.cuda.get_device_name(2)}")
        
        # Set GPU 2 as the default device
        torch.cuda.set_device(2)
        
        # Verify the current device
        current_device = torch.cuda.current_device()
        print(f"Current process is using GPU ID: {current_device}, GPU Name: {torch.cuda.get_device_name(current_device)}")
        
        # Test tensor creation
        test_tensor = torch.randn(10, 10, device=device)
        print(f"Test tensor is on device: {test_tensor.device}")
        
        # Example model placement
        class SimpleModel(torch.nn.Module):
            def __init__(self):
                super().__init__()
                self.linear = torch.nn.Linear(10, 10)
            
            def forward(self, x):
                return self.linear(x)
        
        model = SimpleModel()
        model.to(device)
        print(f"Model is on device: {next(model.parameters()).device}")
    
    else:
        print("Device 2 is not available. Using Device 0.")
        device = torch.device("cuda:0")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

print(f"Final check: Current process is using GPU ID: {torch.cuda.current_device()}, GPU Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")


Set device to: cuda:2
GPU Name: NVIDIA A40
Current process is using GPU ID: 2, GPU Name: NVIDIA A40
Test tensor is on device: cuda:2
Model is on device: cuda:2
Final check: Current process is using GPU ID: 2, GPU Name: NVIDIA A40


In [13]:
from datasets import load_dataset

def load_datasets(dataset1_name, dataset2_name):

    dataset1 = load_dataset(dataset1_name)
    dataset2 = load_dataset(dataset2_name)
    return dataset1, dataset2

def get_unique_urls(dataset1, dataset2):
    """
    Get URLs that are in dataset1 but not in dataset2.
    
    Parameters:
    - dataset1: The first dataset.
    - dataset2: The second dataset.
    
    Returns:
    A list of URLs that are in dataset1 but not in dataset2.
    """
    urls1 = set(dataset1['train']['url'])
    urls2 = set(dataset2['train']['url'])
    
    unique_urls = urls1 - urls2
    return list(unique_urls)

# Example usage
dataset1_name = 'ahmed275/FILTERED_FAST_1'
dataset2_name = 'ahmed275/FILTERED_SLOW'

# Load the datasets
dataset1, dataset2 = load_datasets(dataset1_name, dataset2_name)

# Get the unique URLs
unique_urls = get_unique_urls(dataset1, dataset2)

# Print the unique URLs
print("URLs in dataset1 but not in dataset2:")
for url in unique_urls:
    print(url)


URLs in dataset1 but not in dataset2:
https://www.oyez.org/cases/1993/93-644
https://www.oyez.org/cases/1997/97-428
https://www.oyez.org/cases/1996/95-1340
https://www.oyez.org/cases/1985/84-1362
https://www.oyez.org/cases/1962/490
https://www.oyez.org/cases/1994/93-7901
https://www.oyez.org/cases/1995/95-5257
https://www.oyez.org/cases/2001/00-1770
https://www.oyez.org/cases/1996/94-1988
https://www.oyez.org/cases/1992/91-946
https://www.oyez.org/cases/1992/91-1353
https://www.oyez.org/cases/1993/92-2058
https://www.oyez.org/cases/2002/01-705
https://www.oyez.org/cases/1992/91-522
https://www.oyez.org/cases/1991/91-471
https://www.oyez.org/cases/1994/93-1631
https://www.oyez.org/cases/1997/97-643
https://www.oyez.org/cases/1999/99-51
https://www.oyez.org/cases/1999/99-137
https://www.oyez.org/cases/1991/90-1361
https://www.oyez.org/cases/1994/94-167
https://www.oyez.org/cases/1996/96-552
https://www.oyez.org/cases/1996/96-511
https://www.oyez.org/cases/1991/90-6704
https://www.oyez.or

In [14]:
len(unique_urls)

990

In [15]:
def convert_dataset_to_dataframe(dataset):
    data_opinion = []
    for item in dataset['train']:
        data_opinion.append({
            'id': item['id'],
            'year': item['year'],
            'url': item['url'],
            'opinionOfTheCourt': item['opinionOfTheCourt'],
            'syllabus': item['syllabus'],
            'issueArea': item['issueArea'],
            'decisionDirection': item['decisionDirection']
        })
    
    df = pd.DataFrame(data_opinion)
    return df

In [16]:
from sklearn.model_selection import StratifiedShuffleSplit
from datasets import Dataset
from transformers import LEDTokenizer, LEDForConditionalGeneration, Seq2SeqTrainingArguments, Trainer

df = convert_dataset_to_dataframe(dataset2)


# Create a combined stratification column
df['stratify_col'] = df['issueArea'].astype(str) + '_' + df['decisionDirection'].astype(str)

# Filter out classes with fewer than 2 samples
class_counts = df['stratify_col'].value_counts()
valid_classes = class_counts[class_counts >= 2].index
df = df[df['stratify_col'].isin(valid_classes)]

# Initialize StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, temp_index in split.split(df, df['stratify_col']):
    train_df = df.iloc[train_index]
    temp_df = df.iloc[temp_index]

# Filter out classes with fewer than 2 samples in the temporary dataset
temp_class_counts = temp_df['stratify_col'].value_counts()
valid_temp_classes = temp_class_counts[temp_class_counts >= 2].index
temp_df = temp_df[temp_df['stratify_col'].isin(valid_temp_classes)]

# Further split the temp_df into validation and test sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.6, random_state=42)
for val_index, test_index in split.split(temp_df, temp_df['stratify_col']):
    val_df = temp_df.iloc[val_index]
    test_df = temp_df.iloc[test_index]

# Drop the stratification column
train_df = train_df.drop(columns=['stratify_col'])
val_df = val_df.drop(columns=['stratify_col'])
test_df = test_df.drop(columns=['stratify_col'])

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Print the distribution to verify
print("Training set distribution:")
print(train_df['issueArea'].value_counts(normalize=True))
print(train_df['decisionDirection'].value_counts(normalize=True))

print("\nValidation set distribution:")
print(val_df['issueArea'].value_counts(normalize=True))
print(val_df['decisionDirection'].value_counts(normalize=True))

print("\nTest set distribution:")
print(test_df['issueArea'].value_counts(normalize=True))
print(test_df['decisionDirection'].value_counts(normalize=True))


Training set distribution:
issueArea
1.0     0.218613
8.0     0.202374
2.0     0.172392
9.0     0.105559
3.0     0.093691
7.0     0.056527
4.0     0.043410
12.0    0.042161
10.0    0.040912
6.0     0.012180
5.0     0.012180
Name: proportion, dtype: float64
decisionDirection
2.0    0.533729
1.0    0.466271
Name: proportion, dtype: float64

Validation set distribution:
issueArea
1.0     0.217799
8.0     0.201405
2.0     0.170960
9.0     0.105386
3.0     0.093677
7.0     0.056206
4.0     0.044496
10.0    0.042155
12.0    0.042155
5.0     0.014052
6.0     0.011710
Name: proportion, dtype: float64
decisionDirection
2.0    0.533958
1.0    0.466042
Name: proportion, dtype: float64

Test set distribution:
issueArea
1.0     0.219969
8.0     0.202808
2.0     0.173167
9.0     0.106084
3.0     0.092044
7.0     0.057722
12.0    0.042122
4.0     0.042122
10.0    0.040562
6.0     0.012480
5.0     0.010920
Name: proportion, dtype: float64
decisionDirection
2.0    0.536661
1.0    0.463339
Name: proport

In [19]:
!df -h
!du -sh
!ls -la


Filesystem              Size  Used Avail Use% Mounted on
tmpfs                    26G   13M   26G   1% /run
/dev/mapper/local-root  147G   56G   84G  41% /
tmpfs                   126G   39M  126G   1% /dev/shm
tmpfs                   5.0M  4.0K  5.0M   1% /run/lock
/dev/nvme0n1p1          1.1G  6.1M  1.1G   1% /boot/efi
/dev/mapper/local-home  731G  687G  6.6G 100% /home
/dev/sda1                13T  8.6T  3.5T  72% /srv
tmpfs                    26G  8.0K   26G   1% /run/user/1070
tmpfs                    26G  8.0K   26G   1% /run/user/1023
tmpfs                    26G  8.0K   26G   1% /run/user/1075
tmpfs                    26G  8.0K   26G   1% /run/user/1055
tmpfs                    26G  8.0K   26G   1% /run/user/1076
tmpfs                    26G  8.0K   26G   1% /run/user/1019
tmpfs                    26G  8.0K   26G   1% /run/user/1004
tmpfs                    26G  8.0K   26G   1% /run/user/1057
tmpfs                    26G  8.0K   26G   1% /run/user/1074
1.3G	.
total 1006384
drwx

In [36]:
# Load the tokenizer
tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
i=0
def preprocess_function(examples):
    print(len(examples))
    inputs = examples['opinionOfTheCourt']
    targets = examples['syllabus']
    model_inputs = tokenizer(inputs, max_length=16000, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=1000, truncation=True, padding='max_length')

    print(i+1)

    model_inputs['labels'] = labels['input_ids']

    batch = {}
    batch["input_ids"] = model_inputs.input_ids
    batch["attention_mask"] = model_inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = labels.input_ids

    # We have to make sure that the PAD token is ignored
    # -100 for loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

# Apply the preprocessing function to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/3202 [00:00<?, ? examples/s]

8
1


Map:  31%|███       | 1000/3202 [00:18<00:41, 53.10 examples/s]

8
1


Map:  62%|██████▏   | 2000/3202 [00:36<00:21, 54.84 examples/s]

...

Map:   0%|          | 0/427 [00:00<?, ? examples/s]

8
1


Map:   0%|          | 0/641 [00:00<?, ? examples/s]

8
1


Map: 100%|██████████| 641/641 [00:11<00:00, 55.61 examples/s]


In [2]:
import torch

# Set the device to GPU 2
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda:2


In [37]:
# Load the model
model = LEDForConditionalGeneration.from_pretrained('allenai/led-base-16384')

# Set generate hyperparameters
model.config.num_beams = 2
model.config.max_length = 16000
model.config.min_length = 1000
model.config.length_penalty = 2.0
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Evaluation results:", results)

# Generate summaries
def generate_summary(opinion):
    inputs = tokenizer(opinion, return_tensors='pt', max_length=16383, truncation=True, padding='max_length')
    summary_ids = model.generate(inputs['input_ids'].to("cuda"), max_length=1000, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Test the model on a sample opinion
sample_opinion = df['opinionOfTheCourt'].iloc[0]
print("Opinion:", sample_opinion)
print("Generated Summary:", generate_summary(sample_opinion))

/home/mostah/anaconda3/envs/myenv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Input ids are automatically padded from 16000 to 16384 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 16000 to 16384 to be a multiple of `config.attention_window`: 1024


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/mostah/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mostah/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mostah/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mostah/anaconda3/envs/myenv/lib/python3.11/site-packages/transformers/models/led/modeling_led.py", line 2398, in forward
    outputs = self.led(
              ^^^^^^^^^
  File "/home/mostah/anaconda3/envs/myenv/lib/python3.11/site-packages/t

: 

In [ ]:
df['generated_summary'] = df['opinionOfTheCourt'].apply(generate_summary)

# Prepare data for BERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_bert_function(examples):
    return bert_tokenizer(examples['generated_summary'], padding='max_length', truncation=True, max_length=512)

# Convert the DataFrame to a Dataset
bert_dataset = Dataset.from_pandas(df)

# Apply the preprocessing function to the dataset
tokenized_bert_dataset = bert_dataset.map(preprocess_bert_function, batched=True)

# Split the dataset into train, validation, and test sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, temp_index in split.split(df, df['stratify_col']):
    train_df = df.iloc[train_index]
    temp_df = df.iloc[temp_index]

split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for val_index, test_index in split.split(temp_df, temp_df['stratify_col']):
    val_df = temp_df.iloc[val_index]
    test_df = temp_df.iloc[test_index]

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:

# Load the BERT model
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments for BERT
bert_training_args = TrainingArguments(
    output_dir='./bert_results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer for BERT
bert_trainer = Trainer(
    model=bert_model,
    args=bert_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the BERT model
bert_trainer.train()

# Evaluate the BERT model
bert_results = bert_trainer.evaluate()
print("BERT Evaluation results:", bert_results)


In [ ]:
# Predict decisionDirection using BERT
predictions = bert_trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Calculate the decision direction flip rate
original_labels = test_df['decisionDirection'].values
flip_rate = np.mean(predicted_labels != original_labels)
print("Decision Direction Flip Rate:", flip_rate)

# Calculate the distance-based metric
probabilities = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=-1).numpy()
original_probabilities = np.zeros_like(probabilities)
for i, label in enumerate(original_labels):
    original_probabilities[i, label] = 1

absolute_differences = np.abs(probabilities - original_probabilities)
distance_metric = np.mean(np.sum(absolute_differences, axis=1))
print("Distance-based Metric:", distance_metric)

In [ ]:


# Assuming you have already trained the BERT model and have the test dataset

# Predict decisionDirection using BERT
predictions = bert_trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Extract original labels and group information
original_labels = test_df['decisionDirection'].values
issue_areas = test_df['issueArea'].values

# Define groups based on issueArea
groups = np.unique(issue_areas)

# Initialize dictionaries to store metrics
accuracy = {}
precision = {}
recall = {}
f1 = {}

# Calculate metrics for each group
for group in groups:
    group_indices = np.where(issue_areas == group)[0]
    group_original_labels = original_labels[group_indices]
    group_predicted_labels = predicted_labels[group_indices]
    
    accuracy[group] = accuracy_score(group_original_labels, group_predicted_labels)
    precision[group] = precision_score(group_original_labels, group_predicted_labels, average='macro')
    recall[group] = recall_score(group_original_labels, group_predicted_labels, average='macro')
    f1[group] = f1_score(group_original_labels, group_predicted_labels, average='macro')

# Print metrics for each group
for group in groups:
    print(f"Issue Area {group}:")
    print(f"  Accuracy: {accuracy[group]:.4f}")
    print(f"  Precision: {precision[group]:.4f}")
    print(f"  Recall: {recall[group]:.4f}")
    print(f"  F1 Score: {f1[group]:.4f}")

# Apply the 80% Rule
selection_rates = {group: np.mean(predicted_labels[issue_areas == group] == 1) for group in groups}
max_selection_rate = max(selection_rates.values())
for group, rate in selection_rates.items():
    if rate < 0.8 * max_selection_rate:
        print(f"Group {group} violates the 80% rule with a selection rate of {rate:.4f}")

# Calculate Statistical Parity
positive_rates = {group: np.mean(predicted_labels[issue_areas == group] == 1) for group in groups}
for group, rate in positive_rates.items():
    print(f"Group {group} positive prediction rate: {rate:.4f}")

# Calculate Equalized Odds
true_positive_rates = {group: np.mean((predicted_labels[issue_areas == group] == 1) & (original_labels[issue_areas == group] == 1)) for group in groups}
false_positive_rates = {group: np.mean((predicted_labels[issue_areas == group] == 1) & (original_labels[issue_areas == group] == 0)) for group in groups}
for group in groups:
    print(f"Group {group} true positive rate: {true_positive_rates[group]:.4f}")
    print(f"Group {group} false positive rate: {false_positive_rates[group]:.4f}")

# Visualize Disparities
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
ax[0].bar(groups, [accuracy[group] for group in groups])
ax[0].set_title('Accuracy by Issue Area')
ax[0].set_xlabel('Issue Area')
ax[0].set_ylabel('Accuracy')

ax[1].bar(groups, [precision[group] for group in groups])
ax[1].set_title('Precision by Issue Area')
ax[1].set_xlabel('Issue Area')
ax[1].set_ylabel('Precision')

ax[2].bar(groups, [recall[group] for group in groups])
ax[2].set_title('Recall by Issue Area')
ax[2].set_xlabel('Issue Area')
ax[2].set_ylabel('Recall')

plt.show()
